In [0]:
%run "../01_setup/01_config"


In [0]:
from pyspark.sql import functions as F

### Check if files exist inside landing/claims folder

In [0]:
try:
    files = dbutils.fs.ls(f"{landing_path}/claims")
    csv_files  = [i.path for i in files if i.name.endswith('.csv')]

    if not csv_files:
        print("Not csv files found. Exiting notebook.")
        dbutils.notebook.exit("No Files")

except Exception as e:
    print(f"Error occurred: {e}")
    dbutils.notebook.exit("Error")

### Continue only if files exist inside landing/claims folder

In [0]:
claims_df = spark.read.format('csv')\
    .option('header', True)\
        .load(csv_files)

In [0]:
claims_df = claims_df.withColumn(
    "data_source",
    F.when(F.input_file_name().contains('hospital1'), 'hosa').when(F.input_file_name().contains('hospital2'), 'hosb').otherwise('other')
)

In [0]:
final_df = claims_df\
    .withColumn('ingested_date', F.lit(F.current_timestamp()))\
        .withColumn('Source_file_name', F.input_file_name())\
            .withColumn('is_active', F.lit(True))


In [0]:
final_df.write.mode('append').format('parquet').save(f"{bronze_path}/claims")

### Move the processed files to Archive Folder

In [0]:
from datetime import datetime
date_path = datetime.now().strftime("%Y/%m/%d")

archive_path = f"{landing_path}/archive/claims/{date_path}"

print(f"📦 Archiving processed files to: {archive_path}...")

for file_path in csv_files:
  file_name = file_path.split('/')[-1]
  dest_path = f"{archive_path}/{file_name}"

  dbutils.fs.mv(file_path, dest_path)
  print(f"  -> Moved {file_name}")

print("✅ Processing and Archiving Complete!")